# 5 Big E-Commerce AI Trends

<img src="https://www.persado.com/wp-content/uploads/2023/07/generative-ai-in-e-commerce.jpg" width=800>

Everyone understands that the new era of large scale artificial intelligence models will change the way companies position themselves and their products online, but what research is being done currently, and how specifically are these models being applied. In this notebook, we demonstrate 5 ways that various Generative AI models are being applied to e-commerce settings, covering LLMs, Embedding Models, and Diffusion Models.

**Ideas Outlined:**
1. Virtual Try On Diffusion Models for Trying on Clothes
2. Custom Product Photos Across Various Scenes via Diffusion Models
3. Embedding Model Based Reccomendation Algorithms
4. Language Model Sales Copilots
5. Succinct Product Review Summarization

---
# Virtual Try On via Diffusion Models

Virtual try-on is a technology that allows people to see how clothing or accessories might look on them without physically wearing the items. It uses image generation (diffusion) models to align and adjust the garment images to fit the person's body shape and pose. We have seen examples using Augmented Reality, however this allows a much more realistic generation.

<img src="./media/supreme_jacket_output.png" width=600>

This example was generated using a recently released virtual try on model [CatVTON](https://arxiv.org/abs/2407.15886)- an optimized model based on [Stable Diffusion v1.5 Inpainting](https://huggingface.co/runwayml/stable-diffusion-inpainting), allowing quick and lightweight masked diffusion model generation 

<img src="./media/catvton_diagram.png" width=600>

### [Use Via HuggingFace Spaces](https://huggingface.co/spaces/zhengchong/CatVTON)  
### [Use Via Hosted Gradio Interface](http://120.76.142.206:8888/)

---
# Personalized and Dynamic Product Photos via Diffusion Model Training

While it's easy to take existing photos, cut products out of them, and place them into different scenes. What if we used diffusion models to generate completely novel images of products placed into various scenes? This idea has been explored via different training methods for object permance, namely the [Dreambooth method](https://dreambooth.github.io/). Impressively, this method only requires ~5 photos of an object for training!

### Example 1: Stable Diffusion 3 Medium Dreambooth with a Custom Cube

**Reference:**

<img src="https://cdn-uploads.huggingface.co/production/uploads/65ba68a15d2ef0a4b2c892b4/Fx9Py6PeZL6Jaj4yKk8dl.jpeg" width=300>

**Generations:**

<img src="https://cdn-uploads.huggingface.co/production/uploads/65ba68a15d2ef0a4b2c892b4/UYl02nsBCddvQSB5-egxC.jpeg" width=300>

Model: [AdamLucek/sd3-dreambooth-lora-cube](https://huggingface.co/AdamLucek/sd3-dreambooth-lora-cube)  
Base Model: [stabilityai/stable-diffusion-3-medium-diffusers](https://huggingface.co/stabilityai/stable-diffusion-3-medium-diffusers)  
Training Dataset: [AdamLucek/cube-pics-dreambooth](https://huggingface.co/datasets/AdamLucek/cube-pics-dreambooth)  

### Example 2: Stable Diffusion XL Base Dreambooth with a Green Chair

**Reference:**

<img src="https://cdn-uploads.huggingface.co/production/uploads/65ba68a15d2ef0a4b2c892b4/hvvyMxmnXK36wAEoEPCn2.jpeg" width=300>

**Generations:**

<img src="https://cdn-uploads.huggingface.co/production/uploads/65ba68a15d2ef0a4b2c892b4/tTVfzzi4H09-H8w4dAyA3.jpeg" width=300>

Model: [AdamLucek/sdxl-base-1.0-greenchair-dreambooth-lora](https://huggingface.co/AdamLucek/sdxl-base-1.0-greenchair-dreambooth-lora)  
Base Model: [stabilityai/stable-diffusion-xl-base-1.0](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0)  
Training Dataset: [AdamLucek/green-chair](https://huggingface.co/datasets/AdamLucek/green-chair)  

As we can see, a few limitations of getting the product recreated perfectly every time, but trending positively! With enough tweaking and iterations, can get pretty decent results.

---
# Embedding Model Based Product Reccomendation

<img src="https://miro.medium.com/v2/resize:fit:1078/1*byMr5hdnA4S5WCsQodZ39Q.png" width=600>

Using embedding models, we can rapidly compare products together via vector similarity search. This allows us to compare products together semantically, and can be used as a reccomendation engine, both for similar products or loose descriptions!

For this example, we'll embed 50 different products and show recomendations based on a search and recommendations based on direct product comparison.

In [1]:
import pandas as pd

df = pd.read_csv('/Users/alucek/Documents/Jupyter_Notebooks/ecommerce-trends/home_appliances.csv')
df.head()

,Product Name,Description,Price,Brand,Main Feature,Energy Efficiency Rating
0,Super Blend X1,High-power blender with 10 speed settings and ...,89.99,KitchenMaster,Variable Speed Control,A+
1,Toast-O-Matic 2000,4-slice toaster with bagel and defrost functions,49.99,BreakfastPro,Extra-Wide Slots,A
2,BrewMaster Elite,Programmable coffee maker with 12-cup capacity,79.99,MorningJoe,24-Hour Programmability,A++
3,QuickChef Microwave,1000W microwave with 10 power levels and child...,129.99,QuickChef,One-Touch Cooking,A+
4,FridgeFresh XL,Side-by-side refrigerator with water and ice d...,999.99,CoolTech,In-Door Ice Maker,A+++


In [2]:
import chromadb

# Creating Vector Database
path = "/Users/alucek/Documents/Jupyter_Notebooks/ecommerce-trends/chromadb"
client = chromadb.PersistentClient(path=path)

# Creating VDB Collection to Hold Appliances
appliance_collection = client.get_or_create_collection(name="appliances", metadata={"hnsw:space": "cosine"})

# Putting all appliance descriptions into VDB
for i, row in df.iterrows():
    formatted_info = f"""
        Product Name: {row['Product Name']}
        Description: {row['Description']}
        Price: {row['Price']}
        Brand: {row['Brand']}
        Main Feature: {row['Main Feature']}
        Energy Efficiency Rating: {row['Energy Efficiency Rating']}
        """
    
    appliance_collection.add(
        documents=[formatted_info.strip()],
        ids=[f"appliance_{i}"]
    )

In [4]:
# Similarity Search Function
def appliance_rec(query):
    results = appliance_collection.query(
        query_texts=[query],
        n_results=3)
    print(f"Products Similar to: {query}\n")
    for i in results['documents'][0]:
        print("=========")
        print(f"{i}\n")

# General Search
appliance_rec("I want a coffee maker")
print("---------------------------------")
# Product Name
appliance_rec("FryAir Max")

Products Similar to: I want a coffee maker

Product Name: BrewMaster Elite
        Description: Programmable coffee maker with 12-cup capacity
        Price: 79.99
        Brand: MorningJoe
        Main Feature: 24-Hour Programmability
        Energy Efficiency Rating: A++

Product Name: EspressoPlus Machine
        Description: 15-bar pump espresso machine with milk frother
        Price: 159.99
        Brand: CaffeItaliano
        Main Feature: Dual Thermoblock System
        Energy Efficiency Rating: A

Product Name: CoffeGrind Burr
        Description: Burr coffee grinder with 18 grind settings
        Price: 69.99
        Brand: BeanMaster
        Main Feature: Conical Burr Grinder
        Energy Efficiency Rating: A

---------------------------------
Products Similar to: FryAir Max

Product Name: FryAir Max
        Description: Air fryer with 5.5L capacity and 7 preset cooking functions
        Price: 119.99
        Brand: HealthyCook
        Main Feature: Rapid Air Technology
  

---
# LLM Shopping Copilot

<img src="https://wp.salesforce.com/en-us/wp-content/uploads/sites/4/2023/12/product-features-pdp-plus-gpt-commerce-concierge.jpg?resize=1024,1024" width=600>


Language models can revolutionize the buying experience, offering conversational methods to guide user's towards educating themselves on and purchasing products. This idea has been coined the shopping "[copilot](https://www.salesforce.com/commerce/ai/ecommerce/
)," a chat experience that can answer questions and provide personalized support about complex products.

For this example, we'll create a simple chat sales copilot for the [Nikon Z FC](https://www.nikonusa.com/p/z-fc/1671/overview) mirrorless camera that boast an impressive **638 page user manual**. We'll prompt an LLM to act as a sales expert, and instill the manuals knowledge via a quick RAG setup over the [user manual](https://crossgate.nikonimglib.com/dsd_redirect/redirect.do?P=Pbzwj75&R=mUcXd51&L=F0P1906&O=Ul2WA00).

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Loading and Chunking
loader = PyMuPDFLoader("/Users/alucek/Documents/Jupyter_Notebooks/ecommerce-trends/media/zfc_guide.pdf")
pages = loader.load()

document = ""
for i in range(len(pages)):
    document += pages[i].page_content

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=800,
    chunk_overlap=400,
)

chunks = text_splitter.split_text(document)

In [6]:
# Setting Up Vector Database for QA Context
zfc_collection = client.get_or_create_collection(name="zfc", metadata={"hnsw:space": "cosine"})

In [7]:
# Insert Chunks into ChromaDB Collection
i = 0
for chunk in chunks:
    zfc_collection.add(
    documents=[chunk],
    ids=[f"chunk_{i}"]
    )
    i += 1

In [8]:
SYSTEM_PROMPT = """
You are an AI sales assistant specializing in the Nikon Z fc mirrorless camera. 
Your primary goal is to guide potential customers through their buying journey, providing accurate information, addressing concerns, 
and helping them make an informed decision. Always maintain a friendly, professional, and helpful demeanor.

Key Responsibilities:
1. Provide information about the Nikon Z fc camera, including its features and unique selling points.
2. Assist customers in understanding how the Z fc compares to other Nikon models and competitors' cameras.
3. Offer personalized recommendations based on the customer's needs, preferences, and skill level.
4. Answer questions about compatibility with lenses, accessories, and other Nikon ecosystem products.
5. Explain technical concepts in simple terms for beginners while also being able to engage with more experienced photographers.
6. Guide customers through the purchasing process.
7. Provide after-sales support information, such as warranty details and customer service contacts.

Interaction Guidelines:
- Use a conversational tone while maintaining professionalism.
- Anticipate common questions and concerns about mirrorless cameras and address them proactively.
- Avoid pushy sales tactics; instead, focus on educating and guiding the customer.

Potential Customer Personas:
1. Beginner photographer looking to upgrade from a smartphone
2. Experienced DSLR user considering a switch to mirrorless
3. Vintage camera enthusiast attracted to the retro design
4. Vlogger or content creator interested in video capabilities
5. Travel photographer seeking a compact yet capable camera

"""

In [13]:
import gradio as gr
from typing import List, Tuple
from openai import OpenAI
import json

openai_client = OpenAI()

# Placeholder for your LLM function
def llm_call(hist_messages: List[dict]) -> str:
    
    # Insert historical messages and call GPT-4o for a response
    output = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=hist_messages,
        ) 
    
    content = output.choices[0].message.content
    
    return content

# Placeholder for your RAG retrieval function
def rag_retrieval(query: str) -> str:
    
    # Query ChromaDB collection for top 5 similar entries
    results = zfc_collection.query(
        query_texts=[query],
        n_results=5)
    
    return results

# LLM Logic for determining if RAG is needed
def needs_rag(query: str) -> bool:
    
    # Generate logic via gpt-4o response
    output = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": 'You are determining whether or not more context about a camera product is necessary for a user question.' 
                                          'If it is generic, respond no, if it needs some details, respond yes. Your output should follow the JSON '
                                          'format {"response": "yes"} etc.'},
            {"role": "user", "content": query},
            ],
        ) 
    
    choice = json.loads(output.choices[0].message.content)
    
    if choice['response'].lower() == 'no':
        print("No RAG")
        return False
    else:
        print("RAG Invoked")
        return True

def chat(message: str, history: List[Tuple[str, str]]) -> str:
    # Initialize messages list with the system prompt
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    
    # Add conversation history to messages
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    
    # Check if RAG retrieval is needed
    if needs_rag(message):
        # If RAG is needed, retrieve context and add it as a system message
        context = rag_retrieval(message)
        messages.append({"role": "system", "content": f"Use the following context to answer the user's question: {context}"})
    
    # Add the current user message
    messages.append({"role": "user", "content": message})
    
    # Generate response using the full message history
    response = llm_call(messages)
    
    return response

# Create the Gradio interface
iface = gr.ChatInterface(
    fn=chat,
    chatbot=gr.Chatbot(height=600),
    title="Nikon Z FC LLM Sales Copilot with Full",
    description="Chat with an AI Sales Assistant about the Nikon Z FC Mirrorless Camera. It maintains full conversation context and may use additional retrieval for certain queries.",
    cache_examples=True,
)

# Launch the app
if __name__ == "__main__":
    iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


No RAG
RAG Invoked


In [17]:
# Close Gradio App
iface.close()

Closing server running on port: 7860


---
# LLM Review and Product Summarizations 

<img src="./media/amazon.png" width=600>

Getting reliable information quickly to the user is key with ecommerce, where you want users to make quick decisions, and users want to feel that their decision is thuroughly reliable. We all read reviews of products to base our opinions, but more and more recently I've been noticing summarized reviews- particularly on Amazon. They use a language model to identify the general sentiment and topics being discussed, and give you a hollistic overview. Let's show a way to set that up here

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from IPython.display import display, Markdown

# Setting Up Review Prompt + LLM
review_prompt_template = """
You are an AI assistant specialized in analyzing and summarizing product reviews. 
Your task is to synthesize multiple customer reviews into a concise summary that highlights key points about the product. 
Follow these guidelines:

Start with a brief 'Customers say' section that summarizes the overall sentiment and main points.
Identify 3-5 positive aspects that customers frequently mention.
Identify 2-3 negative aspects or issues that some customers report.
Note any mixed opinions or areas where customer feedback varies.
Create a list of key features or aspects of the product, categorizing them as follows:

Positive (use a green checkmark emoji ✅)
Negative (use a red X emoji ❌)
Mixed opinions (use an orange circle emoji 🟠)

Present the information in a short paragraph no more than 5 sentences long.
Use clear, concise language and avoid unnecessary detail.
If the product is aimed at a specific user group (e.g., beginners), mention this if it comes up frequently in reviews.

Your output should be structured, easy to read at a glance, and provide a balanced view of customer opinions about the product.

----

Example 1 - 3D Printer:
Customers like the value for money, quality, and build quality of the 3D printer. 
They mention it's an excellent printer to start with, produces useful and entertaining things, 
and is a great printer for beginners. However, some customers have reported issues with the build quality, 
bed leveling, and noise level. Opinions are mixed on the print quality, functionality, and ease of assembly.

Value for money ✅
Printer quality ✅
Ease of assembly
Print quality
Functionality
Build quality
Noise level
Bed leveling 🟠

Example 2 - Air Tag Case:

Customers like the cute design, quality, and functionality of the keychain. 
They mention it's well-made, works great with their air tag, and is incredibly functional. 
Some appreciate that it hides the air tag and is value for money. However, some customers 
have different opinions on the size and leather quality.

Looks cute ✅
Quality ✅
Functionality ✅
Airtag holder ✅
Hides air tag ✅
Value for money ✅
Size
Leather quality

Example 3 - Book Shelf:
Customers like the ease of assembly, build quality, and appearance of the shelf. 
They mention it's a solid piece, has good shear strength, and is sturdy to hold their books. 
Customers also say it'll last a long time and is well-designed. They appreciate the size 
and value for money. Customers are also happy with the storage capacity and assembly quality.

Ease of assembly ✅
Build quality ✅
Looks ✅
Size ✅
Value for money ✅
Storage capacity ✅
Assembly quality ✅
Drilled holes 🟠

----

Now for the products and reviews in question

Product Name: {name}

Product Reviews: {reviews}

"""

review_prompt = ChatPromptTemplate.from_template(review_prompt_template)
llm = ChatOpenAI(temperature=0.7, model="gpt-4o")

review_chain = review_prompt | llm | StrOutputParser()

In [15]:
# Loading Reviews for the 'SmartBrew 5000'
df2 = pd.read_csv('/Users/alucek/Documents/Jupyter_Notebooks/ecommerce-trends/reviews.csv')

reviews = df2['Review'].tolist()
name = "SmartBrew 5000"

for i in range(5):
    print(reviews[i], "\n")

The SmartBrew 5000 is a game-changer for my morning routine. I love being able to schedule my coffee from my phone the night before. The taste is great and it's so convenient. However, the app can be a bit glitchy sometimes, which is frustrating when you're half asleep and just want your coffee! 

Absolutely love this coffee maker! The built-in grinder ensures my coffee is always fresh, and the customizable strength settings are perfect. It's a bit pricey, but worth every penny for coffee enthusiasts. The thermal carafe keeps my coffee hot for hours without that burnt taste you get from a hot plate. 

Mixed feelings about the SmartBrew 5000. On one hand, the coffee quality is excellent and I like the smart features. On the other hand, it's quite loud when grinding beans, and the water reservoir is a pain to fill - I always end up spilling water. It's also larger than I expected, taking up a lot of counter space. 

As a tech geek and coffee lover, this is my dream machine. The integrati

In [16]:
# Generating Summary
review_summary = review_chain.invoke({"name": name, "reviews": reviews})
display(Markdown(review_summary))

**Customers say:** The SmartBrew 5000 has garnered mixed reviews, with many praising its coffee quality, smart features, and customizable settings. However, some customers have had issues with the app's functionality, noise levels, and reliability of the machine. It's particularly appreciated by tech enthusiasts and coffee aficionados but may be overly complex for the average user.

**Positive aspects:**
1. High-quality coffee taste
2. Customizable settings, including grind size and brew strength
3. Convenient smart features and app integration
4. Built-in grinder and thermal carafe
5. Large capacity, suitable for offices and large households

**Negative aspects:**
1. App connectivity and functionality issues
2. Noise levels, especially during grinding
3. Reliability concerns, including grinder jams and pump failures

**Mixed opinions:**
1. Complexity and learning curve
2. Size and counter space requirement
3. Maintenance and cleaning process

**Key features:**

- High-quality coffee taste ✅
- Customizable settings ✅
- Convenient smart features ✅
- Built-in grinder ✅
- Large capacity ✅
- App functionality ❌
- Noise levels ❌
- Reliability ❌
- Complexity 🟠
- Size 🟠
- Maintenance 🟠

In summary, the SmartBrew 5000 is a feature-rich coffee maker that excels in producing high-quality coffee with extensive customization options. Its smart features are praised but often marred by app issues and reliability concerns. It's ideal for tech-savvy coffee lovers but might be too complex for those seeking a straightforward coffee-making experience.